In [16]:
# import pandas as pd
# import numpy as np

# file_train_qrels="qrels.dev(1).tsv"
# file_train_queries="Factor40Shrunken9413queries.dev.tsv"
# file_train_bm25_results="runs.msmarco-passage.dev.trec"
# file_passage_lengths = "dev_passage_lengths"

# dfqrels = pd.read_csv(file_train_qrels, sep=" ", names=["query", "/", "document", "relevance"])
# coln = dfqrels.columns.tolist()
# dfqrels = dfqrels[["document","relevance","query"]]

# dfqueries = pd.read_csv(file_train_queries, sep="\t", names=["query", "query text"])

# dfplength = pd.read_csv(file_passage_lengths, sep="\t", names=["document", "length"])


# dfbm25 = pd.read_csv(file_train_bm25_results, sep=" ", names=["query", "/", "document", "rank", "bm25", "/2"])
# def prefixit(x, prefix):
#     return str(prefix)+str(x)

# def prefixqid(x):
#     return prefixit(x, "qid:")

# def prefix1(x):
#     return prefixit(x, "1:")

# def prefix2(x):
#     return prefixit(x, "2:")

In [3]:
import pandas as pd
import numpy as np

file_train_qrels="qrels.dev(1).tsv"
file_train_queries="Factor40Shrunken9413queries.dev.tsv"
file_train_bm25_results="runs.msmarco-passage.dev.trec"

from IPython.display import display, clear_output
# from pyserini.index import IndexReader
display("Gathering QREL File")
qrelfile = open(file_train_qrels, 'r', encoding="utf-8")
clear_output(wait=True)
display("Gathering Query File")
queryfile = open(file_train_queries, 'r', encoding="utf-8")
clear_output(wait=True)
display("Gathering BM25 File")
bm25file = open(file_train_bm25_results, 'r', encoding="utf-8")
clear_output(wait=True)
display("Gathering Collection File")
collectionfile = "collection.tsv"
# clear_output(wait=True)
# display("Reading Index")
# index_reader = IndexReader('indexes/msmarco-passage/lucene-index-msmarco')
clear_output(wait=True)
display("Preparing for Processing")
qrellines = qrelfile.readlines()
querylines = queryfile.readlines()
bm25lines = bm25file.readlines()
passages = pd.read_csv(collectionfile, sep="\t", names=["pid", "passage"]).to_numpy()
qrelfile.close()
queryfile.close()
bm25file.close()
clear_output(wait=True)
display("done")

'done'

In [ ]:
passages[pID][1]

In [ ]:
import random
import itertools

def get_stopwords_ratio(passage):
    stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
    passage_terms = passage.split()
    n_stopwords = 0
    for term in passage_terms:
        if term in stopwords:
            n_stopwords += 1
    return n_stopwords / len(passage_terms)

def get_tf_idf(query, pid):
    tf_dict = index_reader.get_document_vector(str(pid))
    df_dict = {term: (index_reader.get_term_counts(term, analyzer=None))[0] for term in tf_dict.keys()}
    number_of_passages = 8841823
    tf_idf = 0
    query_terms = query.split()
    unique_query_terms = list(dict.fromkeys(query_terms))

    for term in unique_query_terms:
        if term in tf_dict:
            tf = tf_dict[term]
            idf = math.log(number_of_passages / df_dict[term])
            tf_idf += tf * idf
    
    return tf_idf


outFile = open("documentidlist"+str(random.randrange(10000)), 'w', encoding="utf-8")
qrelDict = dict()
queryDict = dict()
queryLengthDict = dict()
maxQL = 0
maxPL = len(max(passages[:,1], key=len))
ResSet = []
display("Gathering relevance judgements")
for qrelLine in qrellines:
    qrelSplit = qrelLine[:-1].split("\t")
    qrelDict[str(qrelSplit[0])+"|"+str(qrelSplit[2])]=qrelSplit[3]
clear_output(wait=True)
display("Gathering queries")
for queryLine in querylines:
    querySplit = queryLine[:-1].split("\t")
    queryDict[querySplit[0]] = querySplit[1]
    QL = len(querySplit[1])
    queryLengthDict[querySplit[0]] = QL
    if (QL > maxQL):
        maxQL = QL
clear_output(wait=True)
display("Processing features")
i = 0
l = 1#len(bm25lines)
for bm25Line in bm25lines:
    bm25Split = bm25Line[:-1].split(" ")
    pID = int(bm25Split[2])
    qID = int(bm25Split[0])
    QID = "qid:" + str(qID)
    passage = passages[pID][1]
    BM25Feature = "1:"+str(bm25Split[4])
#     tfidf = get_tf_idf(queryDict[str(qID)], pID)
    QLFeature = "2:"+str(float(queryLengthDict[str(qID)])/float(maxQL))
    PLFeature = "3:"+str(float(len(passage))/float(maxPL))
    TFIDFFeature = "4:"#+str(tfidf)
    StopWordFeature = "5:"+str(get_stopwords_ratio(passage))
    outFile.write(str(QID) + "\t" + str(BM25Feature)+"\t" + str(QLFeature)+"\t" + str(PLFeature)+"\t" + str(TFIDFFeature)+"\t" + str(StopWordFeature)+"\n")
    if (i % int(l/100) == 0):
        clear_output(wait=True)
        display(str(int(i/l*100))+"%") 
    i+=1
clear_output(wait=True)
display("100%")
outFile.close()
display("Done!")

^C


In [108]:
# dfbm25=dfbm25.loc[dfbm25['rank'] < 11]
# dfbm25['relevance'] = 0
# dfbm25['length'] = 0
# resultArr = dfbm25[['document','relevance','query','bm25', 'length']].to_numpy()

In [ ]:
# i = 0
# for line in resultArr:
#     if (dfqrels[['query','document']].values == [line[2], line[0]]).all(axis=1).any():
#          line[1] = 1
#     l0 = line[0]
#     dfl = dfplength['document']
#     dr = dfplength.loc[(dfl == l0)]
#     try:
#         line[4] = dr['length']
#     except:
#         print(line[0])
#     if (i % 1000 == 0): print(i) 
#     i+=1
# resultArr

In [141]:
# dfresults = pd.DataFrame(data=resultArr, columns=['document','relevance','query','bm25', 'length'])
# dfresults['query'] = dfresults['query'].apply(prefixqid)
# dfresults['bm25'] = dfresults['bm25'].apply(prefix1)
# dfresults['length'] = dfresults['length'].apply(prefix2)
# dfresults

,document,relevance,query,bm25,length
0,4744533.0,0.0,qid:1108939.0,1:1.0,2:0.2001198322348712
1,4069373.0,0.0,qid:1108939.0,1:0.5,2:0.170760934691432
2,3652655.0,0.0,qid:1108939.0,1:0.3333333333333333,2:0.4409826243259437
3,7454708.0,0.0,qid:1108939.0,1:0.25,2:0.13001797483523067
4,841975.0,0.0,qid:1108939.0,1:0.2,2:0.18693828639904134
...,...,...,...,...,...
1995,8522775.0,0.0,qid:532603.0,1:0.16666666666666666,2:0.0
1996,8522782.0,0.0,qid:532603.0,1:0.14285714285714285,2:0.0
1997,8522776.0,0.0,qid:532603.0,1:0.125,2:0.0
1998,2195904.0,0.0,qid:532603.0,1:0.1111111111111111,2:0.0


In [142]:
# dfresults[['relevance','query','bm25', 'length']].to_csv("ranklibconvertoutputtrain.txt",sep="\t", index=False)

In [143]:
# uniqueDocIDs = dfbm25['document'].unique()

In [144]:
# file1 = open("documentidlist", 'w', encoding="utf-8")
# file1.writelines([str(documentID)+"\n" for documentID in uniqueDocIDs])


array([[1185869,       0],
       [1185868,      16],
       [ 597651,      49],
       ...,
       [ 559149, 8841547],
       [ 706678, 8841643],
       [ 405466, 8841735]], dtype=int64)

In [61]:
# len(resultArr)

202180

In [73]:
# resultArr[201157]

array([7.26795500e+06, 0.00000000e+00, 6.13790000e+04, 1.25000000e-01,
       1.09047334e-01])

In [74]:
# resultArr[201158]

array([6.62109500e+06, 0.00000000e+00, 6.13790000e+04, 1.11111111e-01,
       0.00000000e+00])

In [1]:
import wikipedia
import random
import itertools
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

query_file_path="msmarco-test2019-queries.tsv"
query_file = pd.read_csv(query_file_path, sep="\t", names=["qid", "query"])
queries = query_file.to_numpy()
out=""
lines = 1
f = open(str(random.randrange(10000))+'msmarco-test2019-queries-extradesc-n.tsv', 'w', encoding="utf-8")
for query in queries:
    query_text = query[1]
    query_text_splitted = query_text.split()
    stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
    query_text_splitted_reduced = [x for x in query_text_splitted if x not in stopwords]    
    termC = 10
    l=min(termC,len(query_text_splitted_reduced))
    foundAddition = False
    for i in range(l+1,1,-1):
        if not foundAddition:
            for j in range(len(query_text_splitted_reduced)+1-i):
                terms = []
                for k in range(j, j+i):
                    terms.append(query_text_splitted_reduced[k])
                termstring = ' '.join(terms)
#             print(termstring)
                try:
                    summary = wikipedia.summary(termstring, sentences=lines, auto_suggest=True, redirect=True)
                    query_text_splitted.append(summary)
                    out=summary
                    foundAddition = True
                except:
                    out="[ERROR DISAMB]"
#                 clear_output(wait=True)
#                 display("terms:"+termstring+"| i: " + str(i) + "| j: " + str(j) + "| out: " + str(out))
    query_text_rejoined = ' '.join(query_text_splitted)
#     clear_output(wait=True)
    display(str(query[0]query_text + "->")
    display(query_text_rejoined)
    display(" ")
    f.write(str(query[0]) + '\t' + str(query_text_rejoined).replace('\n', ' ').replace('\r', '') + '\n')

print("Done")


'what slows down the flow of blood->'

'what slows down the flow of blood A river is a natural flowing watercourse, usually freshwater, flowing towards an ocean, sea, lake or another river. Hemodynamics or haemodynamics are the dynamics of blood flow.'

' '

'what is the county for grand rapids, mn->'

'what is the county for grand rapids, mn Grand Rapids is a city in and the county seat of Itasca County, Minnesota, United States.'

...

' '

'university of dubuque enrollment->'

'university of dubuque enrollment Clarke University is a private Catholic university in Dubuque, Iowa.'

' '

Done


In [36]:
query_text_splitted

['university', 'of', 'dubuque', 'enrollment']

In [44]:
terms

['university', 'of']

In [43]:
terms.append(query_text_splitted[1])